In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [11]:
%%HTML
<h1> Machine Readable Data Formats </h1>

>
# Recommendations and Best Practices
This python notebook provides practical examples that illustrate the main points discussed in the Guide on machine-readable data. It allows readers to see what are the challenges posed by having data available only in non machine-readable data formats and experience the pitfalls that can cause the generation of invalid files even when using machine-readable data formats like CSV. The notebook also describes how unstructured data, like digital images or other media, can be approached to provide, at least, some pieces of machine-readable information.
Throughout the notebook, examples will be inspired by the dataset "Mounted Specimens of the Historical Bird Collection at the Museum für Naturkunde Berlin" (DOI: [10.7479/wwqn-gd04](https://doi.org/10.7479/wwqn-gd04)). The dataset contains metadata for over 13000 images of mounted bird specimens belonging to the bird collection of the museum. The mounted specimens, which dates back to the 19th century, have now been systematically photographed and their images and related metadata are distributing under a [CC0 Public Domain Dedication](https://creativecommons.org/publicdomain/zero/1.0/deed.en) license. A copy of the dataset is available in the GitHub repository.